In [1]:
# import necessary module
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import seaborn as sns
from array_response import *
import itertools 

import matplotlib.pyplot as plt
import scipy
from scipy.stats import expon,gamma
import scipy.io as io

In [2]:
Nt = 64 # number of transmit antennas
Nr = 16 # number of receive antennas
Ns = 1 # number of streams

Nc = 6 # number of cluster
Nray = 1 # number of rays in each cluster
realization = 1000 # equivalent to number of taking sample

At = np.zeros((Nt,Nc*Nray,realization),dtype=complex)
Ar = np.zeros((Nr,Nc*Nray,realization),dtype=complex)
alpha_hh = np.zeros((Nc*Nray,realization),dtype=complex)
alpha_hv = np.zeros((Nc*Nray,realization),dtype=complex)
alpha_vh = np.zeros((Nc*Nray,realization),dtype=complex)
alpha_vv = np.zeros((Nc*Nray,realization),dtype=complex)
var_hh = np.zeros((Nc*Nray,realization),dtype=float)

AoD = np.zeros((2,Nc*Nray),dtype=complex)
AoA = np.zeros((2,Nc*Nray),dtype=complex)

angle_sigma = 10/180*np.pi # standard deviation of the angles in azimuth and elevation both of Rx and Tx

_gamma = np.sqrt((Nt*Nr)/(Nc*Nray))
count = 0

eps = 0.1 # 20dB isolation
sigma = np.sqrt(8/(1+eps**2)) # according to the normalization condition of H

for reali in range(realization):
    for c in range(1,Nc+1):
        AoD_azi_m = np.random.uniform(0,2*np.pi,1) # Mean Angle of Departure _ azimuth
        AoD_ele_m = np.random.uniform(0,np.pi,1) # Mean Angle of Departure _ elevation
        AoA_azi_m = np.random.uniform(0,2*np.pi,1) # Mean Angle of Arrival_ azimuth
        AoA_ele_m = np.random.uniform(0,np.pi,1) # Mean Angle of Arrival_ elevation
        AoD[0,(c-1)*Nray:Nray*c] = np.random.laplace(AoD_azi_m, angle_sigma, (1,Nray))
        AoD[1,(c-1)*Nray:Nray*c] = np.random.laplace(AoD_ele_m, angle_sigma, (1,Nray))
        AoA[0,(c-1)*Nray:Nray*c] = np.random.laplace(AoA_azi_m, angle_sigma, (1,Nray))
        AoA[1,(c-1)*Nray:Nray*c] = np.random.laplace(AoA_ele_m, angle_sigma, (1,Nray))
    for j in range(Nc*Nray):
        At[:,j,reali] = array_response(AoD[0,j],AoD[1,j],Nt) # UPA array response
        Ar[:,j,reali] = array_response(AoA[0,j],AoA[1,j],Nr)
        var_hh = ((sigma**2)*(np.cos(AoD[0,j])**2)*(np.cos(AoA[0,j])**2)).real
        var_hv = ((eps**2)*(sigma**2)*(np.cos(AoD[1,j])**2)*(np.cos(AoA[0,j])**2)).real
        var_vh = ((eps**2)*(sigma**2)*(np.cos(AoD[0,j])**2)*(np.cos(AoA[1,j])**2)).real
        var_vv = ((sigma**2)*(np.cos(AoD[1,j])**2)*(np.cos(AoA[1,j])**2)).real
        alpha_hh[j,reali] = np.random.normal(0, np.sqrt(var_hh/2)) + 1j*np.random.normal(0, np.sqrt(var_hh/2))
        alpha_hv[j,reali] = np.random.normal(0, np.sqrt(var_hv/2)) + 1j*np.random.normal(0, np.sqrt(var_hv/2))
        alpha_vh[j,reali] = np.random.normal(0, np.sqrt(var_vh/2)) + 1j*np.random.normal(0, np.sqrt(var_vh/2))
        alpha_vv[j,reali] = np.random.normal(0, np.sqrt(var_vv/2)) + 1j*np.random.normal(0, np.sqrt(var_vv/2))

In [3]:
cluster = np.arange(Nc)
print(cluster)
c = list(itertools.combinations(cluster, 2))
num_path = (2*Nc-1)*Nc
path_combi = np.zeros((num_path,4),dtype=int)

print(path_combi.shape)
path_combi[0:Nc,:]=np.arange(Nc).reshape(Nc,1).repeat(4,axis=1)

count = 0
for i in range(int(Nc*(Nc-1)/2)):
    path_combi[Nc+4*i,:] = np.array([c[count][0],c[count][0],c[count][1],c[count][1]])
    path_combi[Nc+4*i+1,:] = np.array([c[count][1],c[count][1],c[count][0],c[count][0]])
    path_combi[Nc+4*i+2,:] = np.array([c[count][0],c[count][1],c[count][1],c[count][0]])
    path_combi[Nc+4*i+3,:] = np.array([c[count][1],c[count][0],c[count][0],c[count][1]])
    count = count+1


[0 1 2 3 4 5]
(66, 4)


In [4]:
cross_index =  []
samepolar_index = []
count = Nc-1
while (count<num_path-4):
    cross_index.extend([count+3,count+4])
    samepolar_index.extend([count+1,count+2])
    count = count + 4
cross_index = np.array(cross_index)
samepolar_index = np.array(samepolar_index)
sameclus_index = np.arange(0,Nc)
print(cross_index)
print(samepolar_index)
print(sameclus_index)

[ 8  9 12 13 16 17 20 21 24 25 28 29 32 33 36 37 40 41 44 45 48 49 52 53 56
 57 60 61 64 65]
[ 6  7 10 11 14 15 18 19 22 23 26 27 30 31 34 35 38 39 42 43 46 47 50 51 54
 55 58 59 62 63]
[0 1 2 3 4 5]


In [5]:
angle_number = 50
path_gain = np.zeros((num_path,realization)) # 2 to save the position and maximum value
gain_sameclus =  np.zeros((realization,angle_number,int(angle_number/2)),dtype=float) # 2 to save the position and maximum value
gain_diffclus_crosspol = np.zeros((realization,angle_number,int(angle_number/2)),dtype=float)
gain_diffclus_samepol = np.zeros((realization,angle_number,int(angle_number/2)),dtype=float)

azi_rot = np.linspace(0,np.pi*2,angle_number)
ele_rot = np.linspace(0,np.pi,int(angle_number/2))
for i in range(angle_number):
    for j in range(int(angle_number/2)):
        for reali in range(realization):
            for combi in range(num_path):
                path_gain[combi,reali] =\
                (np.abs\
                 ((np.cos(ele_rot[j])*np.cos(azi_rot[i])*alpha_hh[path_combi[combi,0],reali]+np.sin(ele_rot[j])*alpha_vh[path_combi[combi,0],reali])*(path_combi[combi,0]==path_combi[combi,1])+\
                 (np.cos(ele_rot[j])*np.cos(azi_rot[i])*alpha_hv[path_combi[combi,2],reali]+np.sin(ele_rot[j])*alpha_vv[path_combi[combi,2],reali])*(path_combi[combi,2]==path_combi[combi,1])+\
                 (-np.sin(ele_rot[j])*np.cos(azi_rot[i])*alpha_hh[path_combi[combi,0],reali]+np.cos(ele_rot[j])*alpha_vh[path_combi[combi,0],reali])*(path_combi[combi,0]==path_combi[combi,3])+\
                 (-np.sin(ele_rot[j])*np.cos(azi_rot[i])*alpha_hv[path_combi[combi,2],reali]+np.cos(ele_rot[j])*alpha_vv[path_combi[combi,2],reali])*(path_combi[combi,2]==path_combi[combi,3])
                 ))**2
                
                gain_sameclus[reali,i,j] = path_gain[sameclus_index[np.argmax(path_gain[sameclus_index,reali])],reali]
                gain_diffclus_crosspol[reali,i,j] = path_gain[cross_index[np.argmax(path_gain[cross_index,reali])],reali]
                gain_diffclus_samepol[reali,i,j] = path_gain[samepolar_index[np.argmax(path_gain[samepolar_index,reali])],reali]
                

In [6]:
gain_sameclus.shape

(1000, 50, 25)

In [7]:
var = {}
var['gain_sameclus']=gain_sameclus
var['gain_diffclus_crosspol']=gain_diffclus_crosspol
var['gain_diffclus_samepol']=gain_diffclus_samepol
var['azi_rot']=azi_rot
var['ele_rot']=ele_rot

io.savemat('gain',var)